In [2]:
#Fine-Tuning BERT with a Logistic Regression Layer
import torch
from torch import nn
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
dataset = load_dataset("imdb")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
import torch
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load IMDB dataset
dataset = load_dataset("imdb")

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Use a smaller subset of the data to speed up training
small_train_dataset = tokenized_datasets["train"].select(range(1000))  # First 1000 samples for training
small_eval_dataset = tokenized_datasets["test"].select(range(500))  # First 500 samples for evaluation

# Define the training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",                 # Output directory
    evaluation_strategy="no",               # Disable evaluation during training
    learning_rate=2e-5,                     # Learning rate
    per_device_train_batch_size=4,          # Smaller batch size for faster training
    per_device_eval_batch_size=4,           # Smaller batch size for evaluation
    num_train_epochs=1,                     # Reduce number of epochs to 1
    fp16=True,                              # Enable mixed precision training (faster on GPUs with Tensor Cores)
    max_steps=1000,                         # Limit the number of steps for faster training
)

# Load the model (using a smaller model like DistilBERT is also an option, but we use BERT here)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Function to extract embeddings (logits) from BERT
def extract_embeddings(model, dataset):
    embeddings = []
    labels = []
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for batch in dataset:
            # Convert the input data to tensors and move them to the device
            input_ids = torch.tensor(batch["input_ids"]).to(model.device)
            attention_mask = torch.tensor(batch["attention_mask"]).to(model.device)
            
            # Ensure the correct shape for the inputs
            input_ids = input_ids.view(input_ids.size(0), -1)  # Flatten if necessary (batch_size, seq_length)
            attention_mask = attention_mask.view(attention_mask.size(0), -1)
            
            labels.append(batch["label"])  # Collect labels
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # We need to extract the logits (which are 2D) and reshape them into 1D for each sample
            logits = outputs.logits.cpu().numpy()  # Shape: (batch_size, num_labels)
            embeddings.extend(logits)  # Append logits to embeddings list, ensuring 2D shape
    # Return both embeddings and labels as 2D and 1D arrays respectively
    return embeddings, labels

# Extract embeddings from the train and test datasets
train_embeddings, train_labels = extract_embeddings(model, small_train_dataset)
test_embeddings, test_labels = extract_embeddings(model, small_eval_dataset)

# Train a logistic regression model on top of the embeddings
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(train_embeddings, train_labels)

# Predict and evaluate
train_preds = log_reg.predict(train_embeddings)
test_preds = log_reg.predict(test_embeddings)

train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

# Initialize the Trainer for BERT
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

# Train the model (BERT fine-tuning)
trainer.train()


/home/j/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
